In [127]:
import pandas as pd
import pprint

In [128]:
lyrics = pd.read_csv('../genius_lyrics21.csv')

Phrase Frequency

In [129]:
docs = lyrics['lyrics'].tolist() #list of Unicode strings

In [130]:
# Tokenize the documents.
from nltk.tokenize import RegexpTokenizer

# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

#Removing stopwords:
from sklearn.feature_extraction import text
sw = ['oh','ooh','yeah','na','la','hey','like','da', 'feat',
      'whoa','uh','huh','doh','doo','ha','eh','ay','ayy','ll','re','ve','ft', 'yo',
     'mmm','nah','ah','woah'] #removing sounds, contractions and featured artists
stop_words= text.ENGLISH_STOP_WORDS.union(sw)

docs = [[token for token in doc if token not in stop_words] for doc in docs]

In [131]:
import nltk

In [132]:
# Lemmatize the documents.
from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [133]:
# Compute bigrams.
from gensim.models import Phrases

# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
phrases = Phrases(docs, min_count=20)

for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [134]:
bgs = pd.Series([x for x in [item for sublist in docs for item in sublist] if '_' in x])
bgs = pd.DataFrame(bgs.value_counts()).reset_index().rename(columns={'index':'phrase',0:'count'})
bgs.head(30)

,phrase,count
0,anderson_paak,722
1,ody_ody,603
2,kidd_keo,567
3,long_live,304
4,young_thug,302
5,mind_mind,272
6,young_stoner,262
7,tryin_tryin,190
8,best_friend,167
9,let_business,156


In [99]:

# #code below is not helpful

# p = []
# s = []

# for phrase, score in phrases.find_phrases(docs).items():
#     p.append(phrase)
#     s.append(score)

# bigrams = pd.DataFrame()
# bigrams['phrase'] = pd.Series(p)
# bigrams['score'] = pd.Series(s)

# bigrams.sort_values('score',ascending=False)